In [1]:
##### MODELING LIBRARIES #####
from tensorflow.keras.models import Sequential
from tensorflow.keras.layers import Dense, Dropout, GRU
from tensorflow.keras.optimizers import Adam
from tensorflow.keras.preprocessing.sequence import TimeseriesGenerator
from tensorflow.random import set_seed
from keras.callbacks import EarlyStopping

from sklearn.preprocessing import StandardScaler
from sklearn.model_selection import train_test_split
from sklearn.metrics import mean_squared_error, r2_score
from patsy import dmatrices, dmatrix

##### FORMATTING AND GRAPHING LIBRARIES #####
import pandas as pd
import numpy as np
import matplotlib.pyplot as plt
import seaborn as sns
import itertools

##### TIMING AND UTILITY LIBRARIES #####
from datetime import datetime
import timeit
from dateutil.relativedelta import relativedelta 
import random

In [2]:
# MAPE function taken from:
# https://www.statology.org/mape-python/
def mape(actual, pred): 
    actual, pred = np.array(actual), np.array(pred)
    return np.mean(np.abs((actual - pred) / actual)) * 100

In [3]:
##### IMPORT PREPARED DATA #####
print('Reading pre-built dataset...')
df_full = pd.read_csv('../PJM_Weekly_Model/sample_base_data.csv', index_col = 0, parse_dates = [0])

# Drop all lag columns for use in RNN
lag_sq_cols = [column for column in df_full.columns if 'Lag' in column]
df_full = df_full.drop(columns = lag_sq_cols)

# Infill remaining nulls (from missing daylight savings hour) with previous value
backfilled = df_full.isna().sum().sum()
print('Backfilling %s null values...'%(backfilled))
df_full = df_full.fillna(method = 'backfill')

# Convert date, time, holiday columns to categorical variables
for col in ['Month','WeekDay','Day','Hour']:
    df_full[col] = df_full[col].astype('category')
    
# Create dataframe for actual loads and predicted train and test values
df_train = pd.DataFrame(df_full['value'])
df_test = pd.DataFrame(df_full['value'])

Reading pre-built dataset...
Backfilling 1 null values...


In [4]:
df_full.columns

Index(['value', 'WWP', 'THI', 'Light', 'WWPSq', 'THISq', 'Month', 'Day',
       'WeekDay', 'Hour', 'Holiday'],
      dtype='object')

In [5]:
# Define final parameters
param_dict = {'seq_length':12,
              'seq_batch_size':256,
              'layer_one_neurons':128,
              'layer_two_neurons':32,
              'layer_three_neurons':16,
              'layer_four_neurons':16,
              'dropout_rate':0,
              'stop_patience':4,
              'learning_rate':0.01,
              'epochs':40
             }

In [9]:
##### RNN ON MOVING TRAINING WINDOW #####
tic = timeit.default_timer()
print("\nDefining training set, building and training models...")

# Define shift variable for taking moving sample of data
data_shift = 0

# while data_shift + 18240 < df_full.shape[0]:
while data_shift == 0 :
    ##### FINAL DATA PREPARATION #####
    
    # Limit dataset to roughly 13 months of data (18240 hours)
    df_modeling = df_full.iloc[data_shift:18240 + data_shift,:].copy()
    
    # Use Patsy to create the one-hot encoded dummy variables with interactions 
    y, X =  dmatrices('value~Light+WWP+THI+WWPSq+THISq+Month+Day+WeekDay+Hour+Holiday',df_modeling,return_type='dataframe')

    # Split data into training and testing data sets with two-year training sample (8760 h/yr * 2)
    X_train, X_test, y_train, y_test = train_test_split(X, y, train_size = 17520 / X.shape[0], shuffle = False)

    # Standardize both datasets - create fit to use on backcast dataset
    ss = StandardScaler()

    X_train_ss = ss.fit_transform(X_train)
    X_test_ss = ss.transform(X_test)
    
    # Limit dataset to roughly 13 months of data (18240 hours)
    df_modeling = df_full.iloc[data_shift:18240 + data_shift,:].copy()

    # Set random seed
    random.seed(283)
    set_seed(283)

    # Create output dataframes for individual predictions
    df_preds_train = pd.DataFrame()
    df_preds_test = pd.DataFrame()

    for x in range(5):
        # Create training sequences
        train_sequences = TimeseriesGenerator(X_train_ss, y_train['value'],
                                              length = param_dict['seq_length'],
                                              batch_size = param_dict['seq_batch_size'])

        # Create test sequences
        test_sequences = TimeseriesGenerator(X_test_ss, y_test['value']
                                             ,length = param_dict['seq_length']
                                             ,batch_size = param_dict['seq_batch_size'])

        ##### RNN MODEL #####
        # Instantiate model and construct all layers
        model = Sequential()
        model.add(GRU(param_dict['layer_one_neurons'], input_shape = (param_dict['seq_length'],77),
                      return_sequences = True, activation = 'relu', ))
        model.add(Dropout(param_dict['dropout_rate']))
        model.add(GRU(param_dict['layer_two_neurons'], return_sequences = True, activation = 'relu'))
        model.add(Dropout(param_dict['dropout_rate']))
        model.add(GRU(param_dict['layer_three_neurons'], return_sequences = True, activation = 'relu'))
        model.add(Dropout(param_dict['dropout_rate']))
        model.add(GRU(param_dict['layer_four_neurons'], return_sequences = False, activation = 'relu'))
        model.add(Dropout(param_dict['dropout_rate']))
        model.add(Dense(1, activation = 'linear'))

        # Compile and fit model
        model.compile(optimizer = Adam(learning_rate = param_dict['learning_rate']),
                      loss = 'mean_squared_error',
                      metrics = 'mean_absolute_percentage_error')

        early_stop = EarlyStopping(monitor='val_loss', min_delta=0, patience = param_dict['stop_patience'], verbose=1, mode='auto')

        model.fit(train_sequences,
                  validation_data = test_sequences,
                  epochs = param_dict['epochs'],
                  callbacks = [early_stop],
                  verbose = True)

        # Write predictions to dataframe, adding nulls equal to lag
        df_preds_train[x] = np.append([np.NaN] * param_dict['seq_length'], model.predict(train_sequences).transpose())
        df_preds_test[x] = np.append([np.NaN] * param_dict['seq_length'], model.predict(test_sequences).transpose())

    # Define name for current model as first date of the testing dataset
    model_date = datetime.date(X_test.index[0])
    
    # Create ensemble prediction and add to final output dataframes
    # NOTE: This would be a good spot for future improvement by creating weighting for the ensemble models
    y_train[model_date] = df_preds_train.mean(axis=1).tolist()
    y_test[model_date] = df_preds_test.mean(axis=1).tolist()
    
    # Output model results
    print('\n%s Model Results' % (model_date))
    
    # Create mask to exclude all records with null values for the give dataset
    mask_train = (y_train[model_date].notna())
    mask_test = (y_test[model_date].notna())
    
    # Calculate r2 scores for training and testing data
    train_r2 = round(r2_score(y_train.loc[mask_train, model_date],y_train.loc[mask_train, 'value']),3)
    test_r2 = round(r2_score(y_test.loc[mask_test, model_date],y_test.loc[mask_test, 'value']),3)
    print('Training data r2: %s' % (train_r2))
    print('Testing data r2: %s' % (test_r2))

    # Calculate MAPE scores for training and testing data
    train_mape = round(mape(y_train.loc[mask_train, model_date],y_train.loc[mask_train, 'value']),3)
    test_mape = round(mape(y_test.loc[mask_test, model_date],y_test.loc[mask_test, 'value']),3)
    print('Training data MAPE: %s' % (train_mape))
    print('Testing data MAPE: %s' % (test_mape))

    # Add the model predictions to the respective dataframes
    df_train = pd.merge(df_train, y_train.drop(columns = 'value'), how = 'left', left_index = True, right_index = True)
    df_test = pd.merge(df_test, y_test.drop(columns = 'value'), how = 'left', left_index = True, right_index = True)
    
    # Increment the data shift by one week (168 hours)
    data_shift += 168

toc = timeit.default_timer()
print('\nAll RNN fits completed in %0.2f seconds' % (toc-tic))


Defining training set, building and training models...
Epoch 1/40
69/69 [==============================] - 3s 39ms/step - loss: 6828176896.0000 - mean_absolute_percentage_error: 82.9582 - val_loss: 6388581888.0000 - val_mean_absolute_percentage_error: 80.9774
Epoch 2/40
69/69 [==============================] - 2s 34ms/step - loss: 2905822720.0000 - mean_absolute_percentage_error: 50.8676 - val_loss: 2544660224.0000 - val_mean_absolute_percentage_error: 48.5641
Epoch 3/40
69/69 [==============================] - 2s 34ms/step - loss: 595330496.0000 - mean_absolute_percentage_error: 21.0658 - val_loss: 1372236800.0000 - val_mean_absolute_percentage_error: 35.5618
Epoch 4/40
69/69 [==============================] - 2s 34ms/step - loss: 365060864.0000 - mean_absolute_percentage_error: 17.2352 - val_loss: 252980384.0000 - val_mean_absolute_percentage_error: 14.2862
Epoch 5/40
69/69 [==============================] - 2s 34ms/step - loss: 169442240.0000 - mean_absolute_percentage_error: 11.27

69/69 [==============================] - 2s 32ms/step - loss: 481012192.0000 - mean_absolute_percentage_error: 18.9373 - val_loss: 1613108224.0000 - val_mean_absolute_percentage_error: 39.8078
Epoch 14/40
69/69 [==============================] - 2s 32ms/step - loss: 544949632.0000 - mean_absolute_percentage_error: 20.0847 - val_loss: 196421568.0000 - val_mean_absolute_percentage_error: 11.6382
Epoch 15/40
69/69 [==============================] - 2s 32ms/step - loss: 139499696.0000 - mean_absolute_percentage_error: 10.0932 - val_loss: 140006736.0000 - val_mean_absolute_percentage_error: 8.8581
Epoch 00015: early stopping
Epoch 1/40
69/69 [==============================] - 3s 37ms/step - loss: 12488180736.0000 - mean_absolute_percentage_error: 78.1598 - val_loss: 454772832.0000 - val_mean_absolute_percentage_error: 16.3243
Epoch 2/40
69/69 [==============================] - 2s 33ms/step - loss: 363372736.0000 - mean_absolute_percentage_error: 16.5468 - val_loss: 555348544.0000 - val_mean

69/69 [==============================] - 2s 29ms/step - loss: 58569280.0000 - mean_absolute_percentage_error: 6.5082 - val_loss: 44174268.0000 - val_mean_absolute_percentage_error: 5.5893
Epoch 16/40
69/69 [==============================] - 2s 29ms/step - loss: 75757472.0000 - mean_absolute_percentage_error: 7.6207 - val_loss: 89064776.0000 - val_mean_absolute_percentage_error: 8.0012
Epoch 17/40
69/69 [==============================] - 2s 29ms/step - loss: 71665072.0000 - mean_absolute_percentage_error: 7.4136 - val_loss: 41039848.0000 - val_mean_absolute_percentage_error: 5.2251
Epoch 18/40
69/69 [==============================] - 2s 29ms/step - loss: 45373720.0000 - mean_absolute_percentage_error: 5.7602 - val_loss: 67033632.0000 - val_mean_absolute_percentage_error: 6.7361
Epoch 19/40
69/69 [==============================] - 2s 29ms/step - loss: 52323536.0000 - mean_absolute_percentage_error: 6.1197 - val_loss: 44251620.0000 - val_mean_absolute_percentage_error: 5.3779
Epoch 20/40


C:\Users\tawha\anaconda3\lib\site-packages\ipykernel_launcher.py:85: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
C:\Users\tawha\anaconda3\lib\site-packages\ipykernel_launcher.py:86: SettingWithCopyWarning: 
A value is trying to be set on a copy of a slice from a DataFrame.
Try using .loc[row_indexer,col_indexer] = value instead

See the caveats in the documentation: https://pandas.pydata.org/pandas-docs/stable/user_guide/indexing.html#returning-a-view-versus-a-copy
